In [1]:
from pennylane import numpy as np
import pennylane as qml
dev = qml.device("default.qubit", wires=[0, 1, "sol"], shots=1)

In [2]:
def find_the_car(oracle):
    """Function which, given an oracle, returns which door that the car is behind.

    Args:
        - oracle (function): function that will act as an oracle. The first two qubits (0,1)
        will refer to the door and the third ("sol") to the answer.

    Returns:
        - (int): 0, 1, 2, or 3. The door that the car is behind.
    """
    @qml.qnode(dev)
    def circuit1():
        # QHACK #
        qml.Hadamard(wires=0)
        qml.PauliX(wires="sol")
        qml.Hadamard(wires="sol")
        oracle()
        qml.Hadamard(wires="sol")
        qml.PauliX(wires="sol")
        qml.Hadamard(wires=0)
        # QHACK #
        return qml.sample()

    @qml.qnode(dev)
    def circuit2():
        # QHACK #
        qml.Hadamard(wires=1)
        qml.PauliX(wires="sol")
        qml.Hadamard(wires="sol")
        oracle()
        qml.Hadamard(wires="sol")
        qml.PauliX(wires="sol")
        qml.Hadamard(wires=1)
        # QHACK #
        return qml.sample()

    sol1 = circuit1()
    sol2 = circuit2()
    # QHACK #

    bit1 = sol1[0]
    bit2 = sol2[1]

    # QHACK #
    if bit1 == 0 and bit2 == 0:
        return 3
    elif bit1 == 1 and bit2 == 1:
        return 0
    elif bit1 == 1 and bit2 == 0:
        return 2
    else:
        return 1

In [3]:
numbers = [0, 0]

def oracle():
    if numbers[0] == 1:
        qml.PauliX(wires=0)
    if numbers[1] == 1:
        qml.PauliX(wires=1)
    qml.Toffoli(wires=[0, 1, "sol"])
    if numbers[0] == 1:
        qml.PauliX(wires=0)
    if numbers[1] == 1:
        qml.PauliX(wires=1)

mapping = {0:3, 1:2, 2:1, 3:0}

output = find_the_car(oracle)
car_door = mapping[numbers[0]*2 + numbers[1]]
print(f"Car is in door {car_door} and we chose {output}")

Car is in door 3 and we chose 3


In [4]:
@qml.qnode(dev)
def visualoracle(numbers):
    if numbers[0] == 1:
        qml.PauliX(wires=0)
    if numbers[1] == 1:
        qml.PauliX(wires=1)
    qml.Toffoli(wires=[0, 1, "sol"])
    if numbers[0] == 1:
        qml.PauliX(wires=0)
    if numbers[1] == 1:
        qml.PauliX(wires=1)
    return qml.sample()
for numbers in [[0,0], [0, 1], [1, 0], [0, 0]]: 
  print(numbers, "car in door number", mapping[numbers[0]*2+numbers[1]])
  drawer = qml.draw(visualoracle)
  print(drawer(numbers))

[0, 0] car in door number 3
  0: ─╭●─┤  Sample
  1: ─├●─┤  Sample
sol: ─╰X─┤  Sample
[0, 1] car in door number 2
  0: ────╭●────┤  Sample
  1: ──X─├●──X─┤  Sample
sol: ────╰X────┤  Sample
[1, 0] car in door number 1
  0: ──X─╭●──X─┤  Sample
  1: ────├●────┤  Sample
sol: ────╰X────┤  Sample
[0, 0] car in door number 3
  0: ─╭●─┤  Sample
  1: ─├●─┤  Sample
sol: ─╰X─┤  Sample


In [5]:
def binary_list(m, n):
    arr = []
    # Create the 0 representation of length n
    for i in range(0, n):
        arr.append(0)
    i = 0
    # Set the elements of arr to represent m
    while m != 0:
        arr[len(arr) - 1 - i] = int(m) % 2
        m = int(m / 2)
        i += 1
    return arr

def get_basis_states(n):
    arr = []
    # Create all possible binary lists from 0 to 2**n
    for i in range(0, 2**n):
        arr.append(binary_list(i, n))
    return arr

def get_state_amplitudes(state):
    i = 0
    m = len(state)
    while m > 1:
        m = m //2
        i += 1
    new_shape = [2]*i
    basis_states = get_basis_states(i)
    state = state.reshape(new_shape)
    for bs in basis_states:
        bs_a = state[bs[0]]
        for i in bs[1:]:
            bs_a = bs_a[i]
        bs_a = np.round(bs_a.real, 3)
        if bs_a != 0:
            st = "|"
            for i in bs:
                st += str(i)
            st += ">"
            print(f"{st}: {bs_a}")
    return st

#get_state_amplitudes(np.array([1]*8))

In [6]:
numbers = [0, 1]

dev2 = qml.device("default.qubit", wires=[0, 1, 2])

def partial_circuit(i):
    if i > 0:
        qml.Hadamard(wires=0)
        qml.PauliX(wires=2)
        qml.Hadamard(wires=2)
    if i > 1:
        if numbers[0] == 1:
            qml.PauliX(wires=0)
        if numbers[1] == 1:
            qml.PauliX(wires=1)
    if i > 2:
        qml.Toffoli(wires=[0, 1, 2])
    if i > 3:
        if numbers[0] == 1:
            qml.PauliX(wires=0)
        if numbers[1] == 1:
            qml.PauliX(wires=1)
    if i > 4:
        qml.Hadamard(wires=2)
        qml.PauliX(wires=2)
        qml.Hadamard(wires=0)


for i in range(6):
    @qml.qnode(dev2)
    def circ():
        partial_circuit(i)
        return qml.state()
    state = circ()
    print(f"state at {i}")
    if i != 0:
        drawer = qml.draw(circ)
        print(drawer()) 
    get_state_amplitudes(state)

state at 0
|000>: 1.0
state at 1
0: ──H────┤  State
2: ──X──H─┤  State
|000>: 0.5
|001>: -0.5
|100>: 0.5
|101>: -0.5
state at 2
0: ──H────┤  State
1: ──X────┤  State
2: ──X──H─┤  State
|010>: 0.5
|011>: -0.5
|110>: 0.5
|111>: -0.5
state at 3
0: ──H────╭●─┤  State
1: ──X────├●─┤  State
2: ──X──H─╰X─┤  State
|010>: 0.5
|011>: -0.5
|110>: -0.5
|111>: 0.5
state at 4
0: ──H────╭●────┤  State
1: ──X────├●──X─┤  State
2: ──X──H─╰X────┤  State
|000>: 0.5
|001>: -0.5
|100>: -0.5
|101>: 0.5
state at 5
0: ──H────╭●──H────┤  State
1: ──X────├●──X────┤  State
2: ──X──H─╰X──H──X─┤  State
|100>: 1.0


In [7]:
numbers = [1, 1]

dev2 = qml.device("default.qubit", wires=[0, 1, 2])

def oracle():
    if numbers[0] == 1:
        qml.PauliX(wires=0)
    if numbers[1] == 1:
        qml.PauliX(wires=1)
    qml.Toffoli(wires=[0, 1, 2])
    if numbers[0] == 1:
        qml.PauliX(wires=0)
    if numbers[1] == 1:
        qml.PauliX(wires=1)

@qml.qnode(dev2)
def circuit1():
    # QHACK #
    qml.Hadamard(wires=0)
    qml.PauliX(wires=2)
    qml.Hadamard(wires=2)
    oracle()
    qml.Hadamard(wires=2)
    qml.PauliX(wires=2)
    qml.Hadamard(wires=0)
    # QHACK #
    return qml.state()

@qml.qnode(dev2)
def circuit2():
    # QHACK #
    qml.Hadamard(wires=1)
    qml.PauliX(wires=2)
    qml.Hadamard(wires=2)
    oracle()
    qml.Hadamard(wires=2)
    qml.PauliX(wires=2)
    qml.Hadamard(wires=1)
    # QHACK #
    return qml.state()

for numbers in [[1,1], [1, 0], [0, 1], [0, 0]]:
    for c in ["c1", "c2"]:
        if c == "c1":
            state = circuit1()
            st = get_state_amplitudes(state)
            print(numbers, c, "bit1=", st[1])
        else:
            state = circuit2()
            st = get_state_amplitudes(state)
            print(numbers, c, "bit2=", st[2])

|100>: -1.0
[1, 1] c1 bit1= 1
|010>: -1.0
[1, 1] c2 bit2= 1
|000>: 1.0
[1, 0] c1 bit1= 0
|010>: 1.0
[1, 0] c2 bit2= 1
|100>: 1.0
[0, 1] c1 bit1= 1
|000>: 1.0
[0, 1] c2 bit2= 0
|000>: 1.0
[0, 0] c1 bit1= 0
|000>: 1.0
[0, 0] c2 bit2= 0
